# Data consistency
What we did so far was looking at each Dataframe individually. Now it's time to look at all Dataframes and see if the information we have is consistent. This include things like

- Are huge price differences explainable with discounts etc. when comparing the prices in the products df and the orders df?
- Is every product ordered present in the products table? 
- Are there significant datetime differences?

If we do not have a solution to inconsistencies that might arise, we'll have to get rid of inconsistent data. At the end of this notebook we want to have merged our tables into one big dataframe, which has all the information we need for our data analysis and visualization. 

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
brands = pd.read_csv('data/brands.csv')
orders = pd.read_csv('data/orders_clean.csv', parse_dates=['created_date'])
orderlines = pd.read_csv('data/orderlines_clean.csv', parse_dates=['date'])
products = pd.read_csv('data/products_clean.csv')

Note that we have to tell pandas which columns should be treated as datetimes. Otherwise the dtype of our columns would have been `object`.

## Inconsistencies regarding the sku

We want to exclude orders which contain products that do not appear in the products list.
Since orderlines also has a sku column, we can compare the orderlines and products df.

In [4]:
orderlines['check_products'] = ~orderlines.sku.isin(products.sku)

Now we should remove every id_order, that contains a product that is not in the list.
Note that every row with such an id_order should be removed, not only the ones with a non-existing sku.
To do this we can create a new table orderlines_id_remove, which is grouped by the id_order and aggregates the False-counts of check_products. If the count is >0, we remove that id_order.
To make life a little easier, I change the check_products column for this: It says True if the sku is NOT in the product list.
Now we can sum up easier.

In [5]:
orderlines_id_remove = orderlines.copy().groupby('id_order').agg({'check_products':'sum'})

We keep only those id_orders, where orderlines_id_remove.check_products == 0.
We also reset the index, so that we don't get a key error in the next step.

In [6]:
orderlines_id_remove = orderlines_id_remove.loc[orderlines_id_remove.check_products == 0].reset_index()

Now we only keep those rows from orderlines, that have an id_order, that appears in orderlines_id_remove

In [7]:
orderlines = orderlines.loc[orderlines.id_order.isin(orderlines_id_remove.id_order)]

Since id_orders should match orders.order_id, we also remove rows from orders and orderlines whose id does not appear in both tables.

In [8]:
orders = orders.loc[orders.order_id.isin(orderlines.id_order)]

In [9]:
orderlines = orderlines.loc[orderlines.id_order.isin(orders.order_id)]

We don't need the column 'check_products' anymore, so it can be dropped.

In [10]:
orderlines.drop(columns=['check_products'], inplace=True)

## Merging tables

We'll now merge the tables at investigate further inconsistencies afterwards.
Since orderlines and products share the sku column and the order_id's from orders and orderlines match, joining these tables don't pose any difficulties. For merging the brands table, we need to create a column with the first 3 characters of the sku.

In [11]:
orderlines['short'] = orderlines.sku.str[:3]
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,short
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,OTT
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,LGE
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,PAR
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,WDT
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,JBL


Now we can create our big dataframe, which we will simply call df.
All the merging could have been done in one step. We split it here for simplicity.

In [12]:
df = pd.merge(orderlines, products, how = 'left', on='sku')
df = df.merge(orders, how='left', left_on='id_order', right_on='order_id')
df = df.merge(brands, how='left', on='short')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290588 entries, 0 to 290587
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                290588 non-null  int64         
 1   id_order          290588 non-null  int64         
 2   product_id        290588 non-null  int64         
 3   product_quantity  290588 non-null  int64         
 4   sku               290588 non-null  object        
 5   unit_price        290588 non-null  float64       
 6   date              290588 non-null  datetime64[ns]
 7   short             290588 non-null  object        
 8   name              290588 non-null  object        
 9   desc              290588 non-null  object        
 10  price             290588 non-null  float64       
 11  promo_price       290588 non-null  float64       
 12  in_stock          290588 non-null  int64         
 13  type              290588 non-null  object        
 14  orde

One thing we notice is that the long column, which stores the brand name, seems to have missing values. 
With some research we could impute the missing values with the help of the short column by hand. It isn't really needed for our analysis later though, since 288 rows do not really have an impact. For now we'll take the 'short' name as the brand.

Then we'll rename columns like `long` to `brand` and drop multiple columns and columns not needed. 

In [14]:
df.rename({'long':'brand', 'product_quantity': 'qty'},axis=1, inplace= True)
df.loc[df.brand.isna(), 'brand'] = df.loc[df.brand.isna(), 'short'] 

In [15]:
df.drop(columns=['order_id', 'promo_price', 'short', 'sku', 'product_id', 'id'], inplace=True)

## Looking at dates
Now that we have our complete dataframe in place, we can start fixing some other issues or remove some outliers that might appear.

We'll first look into the dates. Remember that we have two date columns: 
- `date` from orderlines
- `created_date` from orders

It was already mentioned that a possible explanation for eventual date differences might be: created_date fixes the time when the first step in the ordering process starts (adding items to the shopping basket), while date refers to the time the order was completed (like confirm to buy the selected items)
Following this logic the created_date should always be an earlier point in time. Let's look at the date difference and see if this is true.

In [16]:
df['date_diff'] = df['date']-df['created_date']

In [17]:
df.date_diff.describe()

count                       290588
mean     0 days 04:57:51.118576816
std      6 days 07:46:38.783192706
min            -239 days +04:59:01
25%              -1 days +23:56:42
50%                0 days 00:00:00
75%                0 days 00:00:00
max              397 days 03:11:31
Name: date_diff, dtype: object

It looks like the guess was wrong. Since it isn't clear what the difference is, we'll just remove some outliers (which might be due to some connection/server issues) and afterwards keep only one of the columns.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290588 entries, 0 to 290587
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype          
---  ------        --------------   -----          
 0   id_order      290588 non-null  int64          
 1   qty           290588 non-null  int64          
 2   unit_price    290588 non-null  float64        
 3   date          290588 non-null  datetime64[ns] 
 4   name          290588 non-null  object         
 5   desc          290588 non-null  object         
 6   price         290588 non-null  float64        
 7   in_stock      290588 non-null  int64          
 8   type          290588 non-null  object         
 9   created_date  290588 non-null  datetime64[ns] 
 10  total_paid    290582 non-null  float64        
 11  state         290588 non-null  object         
 12  brand         290588 non-null  object         
 13  date_diff     290588 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(3), int64(3), object(5

In [19]:
from datetime import timedelta, date

In [20]:
df_date_diff = df.groupby('id_order').aggregate({'date_diff':['max', 'min'], 'id_order':'count'}).reset_index()

In [21]:
df_date_diff.describe()

id_order                  date_diff                             \
                                       max                        min   
count 203311.00                     203311                     203311   
mean  412952.13  0 days 05:08:53.290446655  0 days 02:55:30.273880901   
std    65986.70  5 days 21:08:24.044878576  5 days 17:05:12.823665286   
min   241319.00        -217 days +06:24:56        -239 days +04:59:01   
25%   355413.50          -1 days +23:56:25          -1 days +23:56:02   
50%   412832.00            0 days 00:00:00            0 days 00:00:00   
75%   469925.50            0 days 00:00:00            0 days 00:00:00   
max   527401.00          397 days 03:11:31          397 days 03:11:31   

       id_order  
          count  
count 203311.00  
mean       1.43  
std        1.07  
min        1.00  
25%        1.00  
50%        1.00  
75%        1.00  
max      140.00

Here we also see that the time difference does not only happen to orders that consist of more than one product. We'll now save only the `id_order`s with a date_diff of at least one day. We lose about 2% of our orders, but as we might want to do a time analysis we'll rather prioritize time consistency.

In [22]:
id_date_diff = df.loc[abs(df.date_diff) > timedelta(days=1)].id_order.unique().tolist()

In [23]:
df = df.loc[~df.id_order.isin(id_date_diff)]

In [24]:
df = df.drop(columns=['created_date', 'date_diff'])

Actually we are not interested in the exact time, but only the date of the order. So we change the format of the date to %Y-%m-%d:

In [25]:
df['date'] = df['date'].apply(lambda x: x.date())

In [26]:
df.date

0         2017-01-01
1         2017-01-01
2         2017-01-01
3         2017-01-01
4         2017-01-01
             ...    
290583    2018-03-14
290584    2018-03-14
290585    2018-03-14
290586    2018-03-14
290587    2018-03-14
Name: date, Length: 276456, dtype: object

## Prices
Let's look at prices again. Having all of our information in one table we can compare
- What was / is to be paid with what the total price of the bought products is. If we find that much more is paid than the actual cost, we might have corrupted data. 
- price of the products with prices according to the products table. Here we can detect e.g. discounts

To make these types of comparisons, we'll define a new column `total_price` of an order with the help of an auxiliary table for aggregation as well as a `disc_perc` column, which catches the discount compared to the price from the products table in percent.

Furthermore to guarantee consistence we'll make the auxiliary columns 
- `price_prod`: contains the product of all the items' prices of an order. If this value is 0 it means that a product doesn't cost anything. For the sake of reliable data we will drop those orders.
- `payment_diff`: The quotient of `total_paid` and `total_price`. High numbers mean that the payment was way higher than the actual price. We'll get rid of orders which have a suspicuous relation between the payment and the price.

### Prepare the dataframe

In [27]:
df['unit_qty_price'] = df['unit_price'] * df['qty'] #auxiliary column

df['disc_perc'] = (1- (df['unit_price'] / df['price']))*100

In [28]:
df_multi = pd.DataFrame(df.groupby('id_order').agg({'unit_qty_price':['sum', 'prod']})) # auxiliary table
# rename columns and prepare merging 
d = {'sum':'total_price','prod':'price_prod'}
df_multi.columns = df_multi.columns.droplevel(0)
df_multi = df_multi.rename(columns = d).reset_index()

df = pd.merge(df_multi, df, how='right', on='id_order')

In [29]:
df['payment_diff'] = df['total_paid'] / df['total_price'] # auxiliary column

In [30]:
df = df.drop(columns=['unit_qty_price'])

In [31]:
cols = ['id_order', 'brand', 'name', 'desc', 
        'total_paid', 'total_price', 'price', 'unit_price', 'disc_perc', 'qty',
        'date', 'state', 'in_stock', 'type', 'price_prod', 'payment_diff']
df = df[cols]

### Find inconsistencies and drop rows

Time to make our new columns work for us

In [32]:
df = df.loc[~(df.price_prod == 0)]

In [33]:
df = df.loc[~((df.total_paid == 0) & (df.state == 'Completed'))]

It seems unreasonable to pay more for the shipping costs than for the products itself. Because of this we'll remove items where the payment_diff is bigger than 2.

In [34]:
df = df.loc[df.payment_diff < 2]

In [35]:
df.loc[(df.payment_diff > 0) & (df.payment_diff < .9)].sort_values('payment_diff')

,id_order,brand,name,desc,total_paid,total_price,price,unit_price,disc_perc,qty,date,state,in_stock,type,price_prod,payment_diff
247781,506820,Apple,Apple iPhone 64GB Space Gray X,New Free iPhone 64GB X,1.33,1124.00,1159.00,1124.00,3.02,1,2018-02-06,Shopping Basket,1,113271716,1124.00,0.00
251100,508818,Pack,"Apple iMac 27 ""Core i7 Retina 5K 42GHz | 16GB ...",IMac desktop computer 27 inch Retina 5K RAM 16...,6.59,3112.59,3319.00,3112.59,6.22,1,2018-02-09,Shopping Basket,0,"5,74E+15",3112.59,0.00
234497,498550,Apple,"Apple iPad Pro 12.9 ""Wi-Fi + Cellular 512GB Sp...",New iPad Pro 512GB Wi-Fi,3.81,1558.00,1449.00,1407.00,2.90,1,2018-01-25,Shopping Basket,0,51861714,6441246.00,0.00
234498,498550,Apple,Apple Pencil,Pencil Apple iPad Pro.,3.81,1558.00,109.00,109.00,0.00,1,2018-01-25,Shopping Basket,1,1229,6441246.00,0.00
234499,498550,Apple,Apple Lightning to USB 3.0 Adapter,Lightning connector adapter to USB camera,3.81,1558.00,45.00,42.00,6.67,1,2018-01-25,Shopping Basket,1,5395,6441246.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66218,361200,Apple,"Apple MacBook Pro 15 ""Core i7 Touch Bar 29GHz ...",New MacBook Pro 15-inch Core i7 Touch Bar 29Gh...,214642.47,250802.37,3439.00,3266.99,5.00,31,2017-06-01,Shopping Basket,0,2158,71573196569527.25,0.86
153413,442373,Wacom,Wacom Intuos Graphics Tablet S Draw with penci...,Small graphics tablet with pen and Art Rage Li...,64.23,72.99,79.90,72.99,8.65,1,2017-11-25,Shopping Basket,0,1405,72.99,0.88
12279,311414,Moshi,Sensecover Moshi Case iPhone 6 / 6S Black,Cover with touch sensor to answer calls withou...,44.97,49.97,45.00,29.99,33.36,1,2017-01-18,Pending,0,11865403,599.20,0.90
12275,311414,Satechi,Satechi Lightning to USB Cable Flexible 15cm W...,Lightning durable and flexible 15 cm cable len...,44.97,49.97,22.99,9.99,56.55,2,2017-01-18,Pending,1,1230,599.20,0.90


Looking at it from the other perspective, we also see that in some cases only a small amount of the price is actually paid. Of course there are vouchers. But sometime these differences are simply too big. Therefore we say that at least 90% of the total_price has to be paid. This affects about 100 rows so we can live with that. If that number was a lot higher we would have had to think about another solution.

In [36]:
df = df.loc[~((df.payment_diff > 0) & (df.payment_diff < .9))]

Next we'll take a look at the 'disc_perc' column

In [37]:
df.sort_values('disc_perc').head()
#df.sort_values('disc_perc', ascending = False).head(50)

,id_order,brand,name,desc,total_paid,total_price,price,unit_price,disc_perc,qty,date,state,in_stock,type,price_prod,payment_diff
34369,331780,Netatmo,Netatmo home thermostat for iPhone and iPad,Thermostat iPhone iPad wifi Mac design Stark.,159989.83,159989.83,179.00,159989.83,-89279.79,1,2017-03-03,Shopping Basket,0,11905404,159989.83,1.00
71588,366348,Tucano,Tucano SOTTILE ultraslim 8/7 Transparent iPhon...,transparent and flexible cover with 05mm thick...,1741.00,1741.00,7.90,1741.00,-21937.97,1,2017-06-16,Place Order,1,11865403,1741.00,1.00
143679,433935,SwitchEasy,SwitchEasy iPhone Case 8 Plus 0.35 / 7 Plus White,Ultra Thin Case for iPhone 8 Plus / 7 Plus in ...,500.00,500.00,9.99,500.00,-4905.01,1,2017-11-23,Cancelled,0,11865403,500.00,1.00
78343,372580,Service,SSD installation service MacBook Pro Retina,Installing SSD in your MacBook Pro Retina + Da...,1967.61,1967.61,49.99,1967.61,-3836.01,1,2017-07-04,Cancelled,0,20642062,1967.61,1.00
143696,433952,SwitchEasy,SwitchEasy iPhone Case 8 Plus 0.35 / 7 Plus White,Ultra Thin Case for iPhone 8 Plus / 7 Plus in ...,381.02,377.03,9.99,377.03,-3674.07,1,2017-11-23,Cancelled,0,11865403,377.03,1.01
187478,462121,Spek SeeThru,Presidio Speck Case iPhone 8/7 / 6s / 6 Transp...,Protective cover with anti-impact polycarbonat...,816.99,810.00,24.95,810.00,-3146.49,1,2017-12-13,Place Order,0,11865403,810.00,1.01
107171,399589,OWC,SSD expansion kit OWC Aura Pro 6G 1TB Pro Reti...,1TB SSD expansion for MacBook Pro Retina Late ...,756.58,756.58,90.80,756.58,-733.25,1,2017-09-13,Place Order,0,12215397,756.58,1.00
128415,419720,OWC,SSD expansion kit OWC Aura Pro 6G 1TB Pro Reti...,1TB SSD expansion for MacBook Pro Retina Late ...,756.58,756.58,90.80,756.58,-733.25,1,2017-10-30,Shopping Basket,0,12215397,756.58,1.00
137752,428507,OWC,SSD expansion kit OWC Aura Pro 6G 1TB Pro Reti...,1TB SSD expansion for MacBook Pro Retina Late ...,2051.75,2051.75,90.80,756.58,-733.25,1,2017-11-19,Shopping Basket,0,12215397,21174556240.39,1.00
128386,419696,OWC,SSD expansion kit OWC Aura Pro 6G 1TB Pro Reti...,1TB SSD expansion for MacBook Pro Retina Late ...,756.58,756.58,90.80,756.58,-733.25,1,2017-10-30,Place Order,0,12215397,756.58,1.00


We see that there is also a lot going on. It can happen that a person buys a product for a higher price than the price set by the company. But paying almost the price x 90000 seems like a bit too much.
We'll set the threshold to -30, meaning that we'll allow an overpricing of up to 30%. 
Also problems arise the other way around: Sometime we see that a 4000€ MacBook is bought for 7€. This kind of discount seems too much. We'll allow a discount up to 89% (we found some second-hand products in the 88% range, so these seems fine) to remove the most obvious corrupted rows.

To remove the complete orders we collect every id_order first.

In [38]:
id_disc =  df.loc[df.disc_perc < -30].id_order.tolist()
id_disc.extend(df.loc[df.disc_perc > 89].id_order.tolist())

In [39]:
df = df.loc[~df.id_order.isin(id_disc)]

## Save the data
For now our cleaning is finished. We will drop the two auxiliary columns `price_prod` and `payment_diff` we created earlier and save the DataFrame as a csv-file. One can argue that other columns could also be dropped, but since we don't want to unneccessarily restrict ourselves in the visualization part, we'll keep the rest for now.
Categorization was done in another notebook manually. In the future this will be automized using functions.

In [40]:
df = df.drop(columns=['price_prod', 'payment_diff'])
df.to_csv('data/consistent_data.csv')